# WordCount Example

In this WordCount Example our goal is:

&emsp; Take a first look at the Wayang API in a short program
- Wayang API is similar to Spark API
- Do not require "a steep learning curve"

&emsp; Show the provided abstraction on top of Big Data Platforms
- Single Data Analytics task can run on any platform 
- Potentially substasks can run in different Platforms if the task could take advantage of it 

---

## Preparing dependancies


<!--  Importing the needs libraries for the exections, this are comming from the maven instalation perfomed in the step 0 -->

<!-- > This step imports the necessary libraries for subsequent program executions.
All these packages come from the previous Maven Instalation -->
This step imports the required modules to execute the code. All these packages come from the previous Maven Instalation

The imported libraries are:

Module | Java's | Scala's | Description
:----- | -------------: | --------------: | :----------
wayang-core | 8, 11 | 2.11, 2.12 | provides core data structures and the optimizer (required)
wayang-basic | 8, 11 | 2.11, 2.12 | provides common operators and data types for your apps (recommended)
wayang-api-scala-java | 8, 11 | 2.11, 2.12 | provides an easy-to-use Scala and Java API to assemble wayang plans (recommended)
wayang-java | 8, 11 | 2.11, 2.12 | adapters for [Java Stream](https://docs.oracle.com/javase/8/docs/api/java/util/stream/Stream.html) processing platforms
wayang-spark | 8, 11 | 2.11, 2.12 | adapters for [Apache Spark](https://spark.apache.org) processing platforms
wayang-flink | 8, 11 | 2.11, 2.12 | adapters for [Apache Flink](https://flink.apache.org) processing platforms
hadoop-common | 8,11 | - | Hadoop-commons is required because the lack of the Environment Variable **HADOOP_HOME**


In [1]:
/* Import Dependencies */
import $ivy.`org.apache.wayang:wayang-api-scala-java_2.12:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-core:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-basic:0.6.1-SNAPSHOT`
import $ivy.`org.apache.wayang:wayang-java:0.6.1-SNAPSHOT`
import $ivy.`org.apache.hadoop:hadoop-common:2.8.5`
import $ivy.`com.thoughtworks.paranamer:paranamer:2.8`

/* Include required classes */
import org.apache.wayang.api._
import org.apache.wayang.core.api.{Configuration, WayangContext}
import org.apache.wayang.core.util.fs.FileSystems
import org.apache.wayang.java.Java
import java.io.File;

import $ivy.$                                                            

import $ivy.$                                             

import $ivy.$                                              

import $ivy.$                                             

import $ivy.$                                      

import $ivy.$                                         

/* Include required classes */

import org.apache.wayang.api._

import org.apache.wayang.core.api.{Configuration, WayangContext}

import org.apache.wayang.core.util.fs.FileSystems

import org.apache.wayang.java.Java

import java.io.File;

## Wayang Context 

Wayang Context is the entry point for users to work with Wayang. Allows to declare explicitly an execution Platform to run the code.

- The object Java.basicPlugin enables to use Wayang Operators on the Java Platform

> WayangContext also manages the Job creation and its execution as Wayang Plans

In [2]:
val context = new WayangContext().withPlugin(Java.basicPlugin)

context: WayangContext = org.apache.wayang.core.api.WayangContext@4619f668

## Wordcount program - Platform Agnostic

- Plan Builder works as an utility to build and execute Wayang Plans 
- A Wayang Plan consists of a set of operators with dependencies between them
- The provided logical plan is independant of any underlying platform to use
> Wayang performs an optimization process over this Wayang plan, transforming this logical plan in a set of Physical operators to be executable by specific platforms

In [3]:
def wordcount(context: WayangContext) = {
    val planBuilder = new PlanBuilder(context)
    
    val inputFile = new File("book.txt").toURI().toString()
    
    planBuilder
      .withJobName(s"WordCount ($inputFile)")
      .readTextFile(inputFile)
      .flatMap(_.split("\\W+"))
      .filter(_.nonEmpty)
      .map(word => (word.toLowerCase, 1))
      .reduceByKey(_._1, (c1, c2) => (c1._1, c1._2 + c2._2))
      .collect()
}

defined function wordcount

In [4]:
var result = wordcount(context)

result: Iterable[(String, Int)] = Buffer(
  ("frowning", 10),
  ("horsepoliceman", 1),
  ("zulus", 1),
  ("spilling", 3),
  ("auditively", 1),
  ("abrupt", 1),
  ("kraandl", 1),
  ("jellily", 1),
  ("agendath", 14),
  ("salary", 2),
  ("pretend", 5),
  ("childs", 8),
  ("guards", 1),
  ("eamus", 3),
  ("childe", 2),
  ("whenas", 1),
  ("catamite", 1),
  ("widowhood", 1),
  ("stonily", 1),
  ("ruttime", 1),
  ("mulveys", 2),
  ("coals", 5),
  ("avant", 4),
  ("blasts", 1),
  ("forlornlooking", 1),
  ("posthumity", 1),
  ("compassionate", 1),
  ("tallow", 1),
  ("cockhorse", 3),
  ("0", 3),
  ("1", 17),
  ("2", 15),
  ("4", 1),
  ("decorators", 1),
  ("5", 5),
  ("bandolierwise", 1),
  ("6", 11),
  ("tallon", 1),
...

## Including Spark dependencies

Now, Wayang context will reassign the target platform

- First step, adding Wayang Spark Module dependencies

In [5]:

import $ivy.`org.apache.wayang:wayang-spark_2.12:0.6.1-SNAPSHOT`





import $ivy.$                                                   





- Then, import the Spark Class

In [6]:
import org.apache.wayang.spark.Spark

import org.apache.wayang.spark.Spark

- Finally, Wayang context is set to work with Spark Platform

In [7]:
val context = new WayangContext().withPlugin(Spark.basicPlugin)

context: WayangContext = org.apache.wayang.core.api.WayangContext@3596ffea

In [8]:
var result = wordcount(context)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.10/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.30/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
22/03/04 15:07:36 INFO SparkContext: Running Spark version 3.1.2
22/03/04 15:07:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/04 15:07:36 INFO ResourceUtils: ==============================================================
22/03/04 15:07:36 INFO 

result: Iterable[(String, Int)] = Buffer(
  ("celebrant", 3),
  ("mattered", 2),
  ("intimately", 1),
  ("reunion", 2),
  ("bone", 11),
  ("nobleman", 7),
  ("syringes", 1),
  ("showy", 1),
  ("bombast", 1),
  ("alavatar", 1),
  ("runner", 1),
  ("gaping", 11),
  ("overcoats", 4),
  ("gowns", 2),
  ("hem", 2),
  ("spirited", 1),
  ("loveshivery", 1),
  ("been", 203),
  ("fuller", 3),
  ("jade", 1),
  ("jove", 5),
  ("crying", 10),
  ("showtrays", 1),
  ("pwfungg", 1),
  ("breath", 59),
  ("repress", 3),
  ("recurrence", 3),
  ("tongued", 1),
  ("knows", 72),
  ("cleric", 1),
  ("husky", 3),
  ("distrait", 2),
  ("eventuality", 2),
  ("tips", 3),
  ("coylo", 1),
  ("ally", 2),
  ("manuo", 1),
  ("quarts", 1),
...

## Flink Execution

Prepare the dependencies and set the Flink Plugin

In [9]:
import $ivy.`org.apache.wayang::wayang-flink:0.6.1-SNAPSHOT`
import org.apache.wayang.flink.Flink
val context = new WayangContext().withPlugin(Flink.basicPlugin)

import $ivy.$                                               

import org.apache.wayang.flink.Flink

context: WayangContext = org.apache.wayang.core.api.WayangContext@7798c974

In [10]:
var result = wordcount(context)

22/03/04 15:07:43 INFO TypeExtractor: class scala.Tuple2 is missing a default constructor so it cannot be used as a POJO type and must be processed as GenericType. Please read the Flink documentation on "Data Types & Serialization" for details of the effect on performance.
22/03/04 15:07:43 INFO ExecutionEnvironment: The job has 1 registered types and 0 default Kryo serializers


result: Iterable[(String, Int)] = Buffer(
  ("frowning", 10),
  ("zulus", 1),
  ("horsepoliceman", 1),
  ("spilling", 3),
  ("auditively", 1),
  ("kraandl", 1),
  ("abrupt", 1),
  ("jellily", 1),
  ("agendath", 14),
  ("salary", 2),
  ("childs", 8),
  ("pretend", 5),
  ("guards", 1),
  ("eamus", 3),
  ("childe", 2),
  ("catamite", 1),
  ("whenas", 1),
  ("widowhood", 1),
  ("ruttime", 1),
  ("stonily", 1),
  ("mulveys", 2),
  ("coals", 5),
  ("avant", 4),
  ("blasts", 1),
  ("forlornlooking", 1),
  ("posthumity", 1),
  ("tallow", 1),
  ("compassionate", 1),
  ("cockhorse", 3),
  ("0", 3),
  ("1", 17),
  ("2", 15),
  ("4", 1),
  ("5", 5),
  ("decorators", 1),
  ("6", 11),
  ("bandolierwise", 1),
  ("7", 3),
...

## How is the Wordcount in the platforms

### Java Stream Implementation

```java    
public class JavaWordCount {
  public static void main(String args[]) {
    long wordCount = 0;
    Path textFilePath = Paths.get(args[0]);
    try {
      Stream<String> fileLines = 
      Files
          .lines(textFilePath, Charset.defaultCharset())
          .flatMap(line -> Arrays.stream(line.split("\\W+")));
          .filter(t -> t.length() != 0)
          .collect(
              Collectors.toConcurrentMap(
                  w -> w, 
                  w -> 1,
                  Integer::sum
              )
          );
    } catch (IOException ioException) {
      ioException.printStackTrace();
    }
  }
}
```

### Apache Flink Implementation

```scala
object FlinkWordCount {
  def main(args: Array[String]) {
    val env = ExecutionEnvironment.getExecutionEnvironment
    
    env.readTextFile(args(0))
      .flatMap { _.split("\\W+") 
      .filter(t -> t.length() != 0)
      .map { (_, 1) }
      .groupBy(0)
      .sum(1)
      .writeAsCsv(args(1), "\n", " ")
                
    env.execute("FlinkWordCount")
  }
}
```

### Apache Spark Implementation

```scala
object SparkWordCount {
    def main(args: Array[String]) {
      val conf = new SparkConf().setAppName("SparkWordCount")
      val sc = new SparkContext(conf)
      sc.textFile(args(0))
          .flatMap(line => line.split("\\W+"))
          .filter(word => word.length != 0 )
          .map(word => (word, 1)) // implicit convertion to RDDPair
          .reduceByKey{case (x, y) => x + y}
          .saveAsTextFile(args(1))
    }
}
```